In [1]:
!pip install -q tensorflow_datasets

In [8]:
import tensorflow_datasets as tfds

In [9]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']



***************************************************************
Failed to import TensorFlow. Please note that TensorFlow is not installed by default when you install TFDS. This allows you to choose to install either `tf-nightly` or `tensorflow`. Please install the most recent version of TensorFlow, by following instructions at https://tensorflow.org/install.
***************************************************************




ModuleNotFoundError: Failed to construct dataset imdb_reviews: No module named 'tensorflow'

In [7]:
train = train_dataset.take(4000)
test = test_dataset.take(1000)

NameError: name 'train_dataset' is not defined

In [5]:
# to shuffle the data ...
BUFFER_SIZE = 4000 # we will put all the data into this big buffer, and sample randomly from the buffer
BATCH_SIZE = 128  # we will read 128 reviews at a time

train = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)

NameError: name 'train' is not defined

In [ ]:
train = train.prefetch(BUFFER_SIZE)
test = test.prefetch(BUFFER_SIZE)

In [ ]:
VOCAB_SIZE=1000 # assuming our vocabulary is just 1000 words
from tensorflow.keras import layers 
encoder = layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)

encoder.adapt(train.map(lambda text, label: text)) # we just encode the text, not the labels

In [ ]:
# here are the first 20 words in our 1000-word vocabulary
import numpy as np
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

In [ ]:
example, label = list(train.take(1))[0] # that's one batch
len(example)

In [ ]:
example[0].numpy()

In [ ]:
encoded_example = encoder(example[:1]).numpy()
encoded_example

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    encoder, # the encoder
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(layers.LSTM(64)), # making LSTM bidirectional
    tf.keras.layers.Dense(32, activation='relu'), # FC layer for the classification part
    tf.keras.layers.Dense(1) # final FC layer

])

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

In [ ]:
# we will use binary cross entropy again because this is a binary classification task (positive or negative)
# we also did not apply a sigmoid activation function at the last FC layer, so we specify that the 
# are calculating the cross entropy from logits
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    # adam optimizer is more efficient (not always the most accurate though)
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

In [ ]:
model.summary()


In [ ]:
H2 = model.fit(train, epochs=25,
                    validation_data=test)


In [ ]:
import matplotlib.pyplot as plt
plt.plot_results(H2)